In [ ]:
import os
from scipy.io import wavfile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, LSTM
from keras.layers import Dropout, Dense, TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import ModelCheckpoint
import librosa
from tqdm import tqdm

In [ ]:
img_shape = (128, 64, 1) # rows, columns, channels
power = .5
lr = .0002
num_outputs = 3

In [ ]:
data = np.load('./data/ravdess-data.npy', allow_pickle=True)
np.random.shuffle(data)
X_train, y_train = zip(*data)

# reshape
X_train = np.array([x.reshape(img_shape) for x in X_train])

# normalize
_max = np.amax(X_train)
X_train = X_train / _max

# one hot encode target output
y_train = np.array(keras.utils.to_categorical(y_train, num_outputs))

In [ ]:
model = Sequential()
model.add(Conv2D(128, (3, 3), activation='relu', strides=(1, 1), padding='same', input_shape=(img_shape)))
model.add(Conv2D(128, (3,3), activation='relu', strides=(1,1), padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', strides=(1,1), padding='same'))
model.add(MaxPool2D(2,2))
model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dense(num_outputs, activation='softmax'))

adam = keras.optimizers.Adam(lr=lr)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

In [ ]:
history = model.fit(X_train, y_train, verbose=2, epochs=70, batch_size=32, validation_split=0.25)

In [ ]:
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()